<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6b861278501dc90bb4e6fb778dbc62a68e65c8b0996f3bab88f5a2d574771414
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.3
    Uninstalling protobuf-6.33.3:
      Successfully uninstalled protobuf-6.33.3


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-12 16:15:36
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -25.18 K (-0.18%)
Current PnL: -28.28 L (-18.04%)
CY Booked + Current PnL: -13.99 L (-8.92%)
-------------------
Total profit:  1.77 L
Total loss:  -30.05 L
-------------------
Total Booked + Current PnL: 12.99 L (10.01%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.15%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.74 L (66.58%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,1.02,15.70,4.92,21.40,15109.0,41676.0,307102.0,7.34,64.0,X-LC,5.90,14.0,2.76,2.18,23.02,XY25,NTT,FMCG
17,COALINDIA,484.83,3.33,6.31,12.15,19.23,19434.0,9490.0,159951.0,24.27,72.0,L-LC,7.68,182.0,0.49,1.14,28.93,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.05,-22.32,28.86,0.09,22591.0,-22498.0,78279.0,81.64,34.0,M-SC,3.34,253.0,-1.00,0.56,2.84,OX40N,NTT,DURABLES
49,MASFIN,398.61,1.01,-3.93,27.05,22.05,25461.0,-3855.0,94125.0,-17.34,47.0,H-SC,6.28,168.0,-0.15,0.67,36.82,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.23,-14.93,17.58,0.02,26105.0,-26069.0,148490.0,126.56,52.0,M-SC,8.31,240.0,-1.00,1.05,29.16,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,3.33,6.31,12.15,19.23,19434.0,9490.0,159951.0,24.27,72.0,L-LC,7.68,182.0,0.49,1.14,28.93,XY25,ATH,MINING
0,5PAISA,593.00,2.73,-29.73,60.66,12.90,93141.0,-64950.0,153546.0,125.96,60.0,H-SC,9.98,173.0,-0.70,1.09,27.67,OX40N,NTT,FINANCE
37,IEX,219.00,2.65,-3.62,54.20,48.63,104455.0,-7233.0,192721.0,-35.77,51.0,H-SC,13.75,137.0,-0.07,1.37,7.58,XR,NTT,MISC
40,INDUSINDBK,1800.00,2.63,-33.15,98.80,32.89,50992.0,-25596.0,51611.0,-737.61,64.0,L-MC,7.92,259.0,-0.50,0.37,42.13,XR,NTT,BANKS
5,ASIANPAINT,3460.25,2.51,1.24,19.47,20.95,49626.0,3115.0,254883.0,0.21,63.0,X-LC,12.42,36.0,0.06,1.81,37.31,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJESHEXPO,518.00,-3.19,-68.45,217.21,0.07,93995.0,-93903.0,43274.0,1485.86,32.0,L-SC,2.07,269.0,-1.00,0.31,7.63,OX40N,NTT,JEWELLERY
60,REPCOHOME,880.00,-3.00,-12.50,116.94,89.82,293159.0,-35813.0,250692.0,-56.04,46.0,H-SC,4.22,161.0,-0.12,1.78,32.63,XY24,NTT,FINANCE
76,TRIDENT,48.00,-2.75,-31.81,88.46,28.51,55651.0,-29344.0,62911.0,-19.35,32.0,M-SC,7.76,226.0,-0.53,0.45,7.65,XR,NTT,TEXTILES
9,BAJAJHLDNG,14451.52,-2.70,-3.92,34.50,29.22,63019.0,-7460.0,182665.0,-6.67,36.0,X-LC,11.74,29.0,-0.12,1.30,4.90,X40,ATH,FINANCE
15,CAMS,950.00,-2.31,-6.91,33.71,24.48,79038.0,-17391.0,234465.0,-82.21,34.0,X-SC,3.65,86.0,-0.22,1.67,14.64,X40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,1.9,0.3,20.38,20.74,41529.0,605.0,203775.0,33.49,57.0,M-MC,5.46,189.0,0.01,1.45,10.94,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.81,-4.16,116.94,107.91,165474.0,-6141.0,141503.0,-22.17,45.0,M-SC,10.11,220.0,-0.04,1.01,5.51,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.23,-14.93,17.58,0.02,26105.0,-26069.0,148490.0,126.56,52.0,M-SC,8.31,240.0,-1.00,1.05,29.16,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.04,-17.95,60.90,32.03,65879.0,-23659.0,108176.0,-11.17,64.0,H-SC,6.82,157.0,-0.36,0.77,21.17,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-2.05,-22.32,28.86,0.09,22591.0,-22498.0,78279.0,81.64,34.0,M-SC,3.34,253.0,-1.00,0.56,2.84,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.78,-23.03,47.42,13.47,98433.0,-62091.0,207576.0,-68.31,45.0,H-SC,4.52,159.0,-0.63,1.47,5.55,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,1.90,0.30,20.38,20.74,41529.0,605.0,203775.0,33.49,57.0,M-MC,5.46,189.0,0.01,1.45,10.94,XY25,NTT,FINANCE
17,COALINDIA,484.83,3.33,6.31,12.15,19.23,19434.0,9490.0,159951.0,24.27,72.0,L-LC,7.68,182.0,0.49,1.14,28.93,XY25,ATH,MINING
84,WIPRO,420.00,0.44,7.91,59.64,72.27,109055.0,13407.0,182855.0,-7.44,51.0,M-LC,6.19,101.0,0.12,1.30,14.58,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.44,7.91,59.64,72.27,109055.0,13407.0,182855.0,-7.44,51.0,M-LC,6.19,101.0,0.12,1.30,14.58,XR,NTT,IT
25,FINCABLES,1641.55,-1.81,-4.16,116.94,107.91,165474.0,-6141.0,141503.0,-22.17,45.0,M-SC,10.11,220.0,-0.04,1.01,5.51,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.39,-7.45,123.36,106.72,140817.0,-9185.0,114151.0,-54.25,36.0,H-SC,7.63,138.0,-0.07,0.81,14.19,AR,ATH,MISC
37,IEX,219.00,2.65,-3.62,54.20,48.63,104455.0,-7233.0,192721.0,-35.77,51.0,H-SC,13.75,137.0,-0.07,1.37,7.58,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,0.02,-5.62,43.70,35.63,86441.0,-11768.0,197806.0,-18.95,36.0,H-MC,3.33,119.0,-0.14,1.41,10.12,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.37,-17.06,33.57,10.78,66911.0,-41000.0,199318.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.37,X40,NTT,FMCG
7,AWL,485.00,-1.42,-28.75,120.64,57.19,280307.0,-93777.0,232350.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.65,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.07,0.25,-13.02,50.17,30.62,103446.0,-30864.0,206190.0,-38.20,27.0,X-MC,16.66,57.0,-0.30,1.46,0.25,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.76,-41.31,130.37,35.20,98453.0,-53152.0,75518.0,-0.94,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
78,UNITDSPR,1644.00,-0.26,-2.48,23.83,20.77,54415.0,-5797.0,228347.0,-10.93,30.0,X-MC,1.31,70.0,-0.11,1.62,4.27,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.26,-2.48,23.83,20.77,54415.0,-5797.0,228347.0,-10.93,30.0,X-MC,1.31,70.0,-0.11,1.62,4.27,X40N,NTT,BREWERIES
45,JIOFIN,387.00,0.35,-7.28,34.26,24.48,84929.0,-19470.0,247895.0,-8.80,39.0,X-LC,1.54,37.0,-0.23,1.76,43.69,XY24,BTT,FINANCE
65,SIEMENS,4671.50,-2.07,-19.91,56.72,25.51,84535.0,-37055.0,149040.0,-8.57,37.0,H-LC,1.95,50.0,-0.44,1.06,9.83,AR,ATH,ELECTRICAL
81,VBL,671.64,0.98,-0.25,36.01,35.67,113447.0,-798.0,315044.0,-12.17,56.0,X-LC,2.29,5.0,-0.01,2.24,13.57,X40N,ATH,FMCG
20,DABUR,735.00,0.31,2.45,40.27,43.70,102342.0,6072.0,254140.0,-5.95,67.0,X-MC,2.34,73.0,0.06,1.81,17.87,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.42,-28.75,120.64,57.19,280307.0,-93777.0,232350.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.65,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.03,-19.92,94.14,55.47,183416.0,-48453.0,194833.0,-29.11,33.0,X-MC,15.70,63.0,-0.26,1.38,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.76,-41.31,130.37,35.20,98453.0,-53152.0,75518.0,-0.94,27.0,X-SC,17.48,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-1.06,-7.11,44.46,34.19,88180.0,-15184.0,198336.0,-34.66,32.0,X-MC,4.09,60.0,-0.17,1.41,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-1.62,-49.00,198.63,52.31,147054.0,-71126.0,74034.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.61,-28.64,100.16,42.84,46465.0,-18615.0,46391.0,-53.17,48.0,X-SC,26.52,83.0,-0.40,0.33,4.95,XY24,NTT,MISC
58,RELAXO,1176.00,-1.62,-49.00,198.63,52.31,147054.0,-71126.0,74034.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.76,-41.31,130.37,35.20,98453.0,-53152.0,75518.0,-0.94,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,1.57,-0.32,24.48,24.08,30978.0,-407.0,126543.0,-8.38,50.0,X-SC,6.13,91.0,-0.01,0.90,19.23,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-1.00,-18.55,75.96,43.32,100769.0,-30212.0,132660.0,-30.03,39.0,X-SC,7.97,90.0,-0.30,0.94,1.86,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.37,-17.06,33.57,10.78,66911.0,-41000.0,199318.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.37,X40,NTT,FMCG
75,TMPV,600.00,-1.02,-28.15,71.16,22.98,115496.0,-63593.0,162305.0,-25.52,38.0,X-LC,4.10,2.0,-0.55,1.15,2.08,XY24,NTT,AUTO
73,TCS,4389.97,0.99,-10.92,35.51,20.71,119640.0,-41319.0,336918.0,-23.30,52.0,X-LC,2.36,3.0,-0.35,2.39,12.58,X40,ATH,IT
81,VBL,671.64,0.98,-0.25,36.01,35.67,113447.0,-798.0,315044.0,-12.17,56.0,X-LC,2.29,5.0,-0.01,2.24,13.57,X40N,ATH,FMCG
41,INFY,1972.00,-1.13,8.64,23.57,34.24,80121.0,27039.0,339927.0,-15.29,46.0,X-LC,7.63,6.0,0.34,2.41,17.64,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.63,-33.15,98.80,32.89,50992.0,-25596.0,51611.0,-737.61,64.0,L-MC,7.92,259.0,-0.50,0.37,42.13,XR,NTT,BANKS
69,SURYODAY,216.00,0.36,-18.49,50.95,23.03,74363.0,-33119.0,145952.0,60.09,55.0,H-SC,3.96,165.0,-0.45,1.04,44.81,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.23,-14.93,17.58,0.02,26105.0,-26069.0,148490.0,126.56,52.0,M-SC,8.31,240.0,-1.00,1.05,29.16,OX40N,NTT,PAINTS
17,COALINDIA,484.83,3.33,6.31,12.15,19.23,19434.0,9490.0,159951.0,24.27,72.0,L-LC,7.68,182.0,0.49,1.14,28.93,XY25,ATH,MINING
84,WIPRO,420.00,0.44,7.91,59.64,72.27,109055.0,13407.0,182855.0,-7.44,51.0,M-LC,6.19,101.0,0.12,1.30,14.58,XR,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,2.73,-29.73,60.66,12.90,93141.0,-64950.0,153546.0,125.96,60.0,H-SC,9.98,173.0,-0.70,1.09,27.67,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,0.04,-17.95,60.90,32.03,65879.0,-23659.0,108176.0,-11.17,64.0,H-SC,6.82,157.0,-0.36,0.77,21.17,OX40N,NTT,IT
17,COALINDIA,484.83,3.33,6.31,12.15,19.23,19434.0,9490.0,159951.0,24.27,72.0,L-LC,7.68,182.0,0.49,1.14,28.93,XY25,ATH,MINING
57,RECLTD,446.00,1.90,0.30,20.38,20.74,41529.0,605.0,203775.0,33.49,57.0,M-MC,5.46,189.0,0.01,1.45,10.94,XY25,NTT,FINANCE
56,RAJOOENG,143.10,0.03,-33.68,110.50,39.61,75118.0,-34520.0,67980.0,-55.07,49.0,H-SC,30.28,136.0,-0.46,0.48,10.04,AR,ATH,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.44
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.51
MC    30.79
LC    26.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.24
X40      24.02
X40N     12.78
XY25     10.89
XR        8.98
AR        8.53
OX40N     7.64
SR        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
X-LC    23.22
H-SC    22.75
M-SC    11.20
X-SC     7.79
H-MC     4.60
M-MC     1.45
M-LC     1.30
L-LC     1.14
H-LC     1.06
L-SC     0.77
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-8.73,44.41
IT,12.88,-17.81,77.24
FINANCE,11.04,-14.71,62.85
MISC,7.03,-31.35,85.09
ELECTRICAL,5.88,-13.27,54.76
PAINTS,5.80,-12.95,29.73
INSURANCE,4.74,-1.80,36.64
PHARMA,3.98,-4.98,38.02
BANKS,2.78,-37.08,120.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3277529.0,21
AR,1370293.0,10
XR,1228859.0,12
X40,1138611.0,15
X40N,943864.0,9
OX40N,718012.0,10
XY25,409328.0,7
SR,287305.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3590652.0,24
X-MC,1602661.0,16
M-SC,1483531.0,15
X-LC,953864.0,13
X-SC,823649.0,8
H-MC,431699.0,3
L-SC,182200.0,2
M-LC,109055.0,1
H-LC,84535.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270547.0      6
           AR         928142.0      5
M-SC       XY24       850915.0      6
H-SC       XR         792289.0      7
X-MC       X40        519921.0      7
           XY24       451541.0      3
X-LC       X40        419270.0      6
X-MC       X40N       410012.0      4
X-SC       X40N       326861.0      3
H-SC       OX40N      312369.0      4
M-SC       OX40N      311648.0      5
X-SC       XY24       297368.0      3
H-SC       SR         287305.0      2
H-MC       AR         224966.0      2
X-MC       XY25       221187.0      2
X-LC       X40N       206991.0      2
H-MC       XY24       206733.0      1
X-LC       XY24       200425.0      2
X-SC       X40        199420.0      2
M-SC       XR         188318.0      2
           AR         132650.0      2
X-LC       XY25       127178.0      3
M-LC       XR         109055.0      1
L-SC       OX40N       93995.0      1
           XR          88205.0      1
H-LC       AR          84535.0      1
L-MC       XR          50992.0      1
M-MC       XY25        41529.0      1
L-LC       XY25        19434.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
